## A Widget for Inspecting Highlighting
Manually testing syntax highlighting is one approach for catching regressions. This widget will look at a token file created by `Get Cell Source Tokens`, and compare it with a known-good fixture.

If it finds any difference, it will show a link to the sample in question which will pop open a new editor in JupyterLab. If you manually verify the output, you can `Trust` the sample and copy the token file into the expected location.

### So What?
While this may not be _immediately_ useful to you unless you are writing a syntax highlighter, the pattern is useful for all kinds of automation/testing workflows:

- write a test that generates some structured output
- compare the output to a known good example (if available)
- provide a way to incrementally (and interactively!) for a person to improve the fixture data
- repeat the test until you're done

In [ ]:
import shutil, re, traitlets as T, IPython.display as D, ipywidgets as W
from pathlib import Path
from difflib import context_diff

In [ ]:
class SyntaxChecker(W.Widget):
    sample_root = T.Instance(Path)
    sample_glob = T.Unicode()
    trusted_root = T.Instance(Path)
    observed_root = T.Instance(Path)
    blacklist = T.Tuple()

    samples = T.Dict()
    trusted = T.Dict()
    observed = T.Dict()
    disagree = T.Dict()
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._ndlink(["sample_root", "sample_glob"], "samples", self._update_samples)
        self._ndlink(["observed_root"], "observed", self._update_observed)
        self._ndlink(["trusted_root"], "trusted", self._update_trusted)
        self._ndlink(["trusted", "observed"], "disagree", self._update_disagree)
    
    def _ndlink(self, src, dest, handler):
        [T.dlink((self, s), (self, dest), handler) for s in src]
    
    def _rel_stem(self, path, root):
        return str(path.relative_to(root))[:-len(path.suffix)]
    
    def _stem_dict(self, root, pattern):
        if root and pattern:
            return {
                self._rel_stem(p, root): p
                for p in root.glob(pattern)
                if all([b not in str(p) for b in self.blacklist])
            }
        else:
            return {}
        
    
    def _update_samples(self, _):
        return self._stem_dict(self.sample_root, self.sample_glob)
    
    def _update_observed(self, _):
        return self._stem_dict(self.observed_root, "**/*.tokens")
    
    def _update_trusted(self, _):
        return self._stem_dict(self.trusted_root, "**/*.tokens")
    
    def _update_disagree(self, _):
        disagree = {}
        for k, v in self.observed.items():
            observed = v.read_text()
            if k in self.trusted:
                trusted = self.trusted[k].read_text()
            else:
                trusted = ""
            if observed != trusted:
                disagree[k] = "\n".join(
                    context_diff(observed.split("\n"), trusted.split("\n"))
                )
        return disagree
    
    def _disagree_row(self, k, v):
        link = W.Output()
        btn = W.Button(description="Trust?", button_style="primary")
        
        @link.capture()
        def _show():
            D.display(D.Markdown(f"[{k}]({self.samples[k]})"))
        
        def _click(btn):
            if btn.description == "Trust?":
                btn.description = "Confirm?"
                btn.button_style = "warning"
            elif btn.description == "Confirm?":
                observed = self.observed[k]
                trusted = self.trusted_root / observed.relative_to(self.observed_root)
                trusted.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy(observed, trusted)
                btn.description = "Trusted!"
                btn.button_style = "success"
                
        btn.on_click(_click)
        
        _show()
        return W.HBox([btn, link, 
                       W.HTML(f"<details><summary>{len(v)} changes</summary><pre>{v}</pre></details>")
                      ])
    
    def show_disagree(self):
        return W.VBox([W.HTML("<h2>Untrusted Samples</h2>")] + ([
            self._disagree_row(k, v) for k, v in self.disagree.items()
        ] or [W.HTML("<h3>Nothing to report!</h3>")]))

Not sure if this is actually importable yet, but for now gate behind a 

In [ ]:
if __name__ == "__main__":
    try: del checker
    except: pass
    checker = SyntaxChecker(
        sample_root=Path("..") / "fixtures" / "highlighting" / "samples",
        sample_glob="**/*.robot",
        trusted_root=Path("..") / "fixtures" / "highlighting" / "tokens",
        observed_root=Path("../..") / "_testoutput" / "headlessfirefox" / "tokens",
        blacklist=["ipynb_checkpoints"]
    )
    try: del disagree
    except: pass
    disagree = checker.show_disagree()
    D.display(disagree)